# 1) Install and import dependencies:

In [1]:
!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch torchaudio torchvision torchtext torchdata

Found existing installation: torch 2.3.0+cu121
Uninstalling torch-2.3.0+cu121:
  Successfully uninstalled torch-2.3.0+cu121
Found existing installation: torchaudio 2.3.0+cu121
Uninstalling torchaudio-2.3.0+cu121:
  Successfully uninstalled torchaudio-2.3.0+cu121
Found existing installation: torchvision 0.18.0+cu121
Uninstalling torchvision-0.18.0+cu121:
  Successfully uninstalled torchvision-0.18.0+cu121
Found existing installation: torchtext 0.18.0
Uninstalling torchtext-0.18.0:
  Successfully uninstalled torchtext-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 78.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64

In [2]:
!pip install transformers requests beautifulsoup4

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [44]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

# 2) Instantiate the model:

In [5]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3) Encode and calculate sentiment:

In [75]:
# pt = pytorch tensors, can put tf for tensorflow
tokens = tokenizer.encode('I love you', return_tensors = "pt")
tokens

tensor([[  101,   151, 11157, 10855,   102]])

In [76]:
tokens

tensor([[  101,   151, 11157, 10855,   102]])

In [77]:
tokenizer.decode(tokens[0])

'[CLS] i love you [SEP]'

In [78]:
result = model(tokens)
result #logits = output class probabilities

SequenceClassifierOutput(loss=None, logits=tensor([[-2.0087, -2.1010, -0.6899,  1.0339,  3.0444]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [79]:
torch.argmax(result.logits)

tensor(4)

In [80]:
# convert to integer value
# 1 star = negative sentiment, 5 star = positive sentiment

int(torch.argmax(result.logits)) + 1

5

# 4) collect reviews for sentiment:

In [34]:
r = requests.get("https://www.yelp.com/biz/mejico-sydney-2")
r.text

'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>\n            window.yelp = window.yelp || {};\

In [36]:
soup = BeautifulSoup(r.text, "html.parser")

In [42]:
regex = re.compile(".*comment.*")
results = soup.find_all('p', {"class" : regex})
results[0] # not adding .text gives us <p> tag information which we don't need, hence we add .text

<p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future.</span></p>

In [43]:
reviews  = [result.text for result in results]
reviews[0]

"Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future."

#5) Load reviews into dataframe and score them:

In [47]:
reviews[:5]

["Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future.",
 'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to che

In [48]:
np.array(reviews[:5])

array(["Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future.",
       'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even

In [49]:
 # datadrame with or without np.array, would recommened putting it in when working with numeric data

df = pd.DataFrame(np.array(reviews), columns = ['review'])
df.head()

,review
0,Seated without a booking on a super busy Satur...
1,The food was decent not great.. We had the gu...
2,"Food was okay, guacamole was below average. Se..."
3,The food and service here was really good. It...
4,Visiting from Texas and decided to give this r...


In [50]:
# we will loop through all reviews and pass them through model and get sentiment score:

df.shape

(10, 1)

In [51]:
def get_sentiment_score(text):
  tokens = tokenizer.encode(text, return_tensors = "pt")
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1

In [52]:
get_sentiment_score("love the movie")

5

In [83]:
df['sentiment'] = df['review'].apply(lambda x: get_sentiment_score(x[:512])) # keep a token limit of 512
df.head()

,review,sentiment
0,Seated without a booking on a super busy Satur...,5
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Visiting from Texas and decided to give this r...,5


# 6) Putting it all together:

In [92]:
def get_reviews_df(link):
  r = requests.get(link)
  soup = BeautifulSoup(r.text, "html.parser")
  regex = re.compile(".*comment.*")
  results = soup.find_all('p', {"class" : regex})
  reviews =  [result.text for result in results]
  df = pd.DataFrame(np.array(reviews), columns = ['review'])
  return df

def get_sentiment_score(text):
  tokens = tokenizer.encode(text, return_tensors = "pt")
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1

In [94]:
reviews_df = get_reviews_df("https://www.yelp.com/biz/mejico-sydney-2")
reviews_df['sentiment'] = reviews_df['review'].apply(lambda x: get_sentiment_score(x[:512]))
reviews_df

,review,sentiment
0,Seated without a booking on a super busy Satur...,5
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Visiting from Texas and decided to give this r...,5
5,Don't come here expecting legit Mexican food b...,3
6,Out of all the restaurants that I tried in Syd...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,The food is fresh and tasty. The scallop cevi...,4


# 7) Trying other businesses:

In [95]:
reviews_df = get_reviews_df("https://www.yelp.com/biz/flying-fajita-sistas-glebe?page_src=related_bizes")
reviews_df['sentiment'] = reviews_df['review'].apply(lambda x: get_sentiment_score(x[:512]))
reviews_df

,review,sentiment
0,This was a neighborhood restaurant stop for so...,4
1,Excellent sangrias! We tried the special sangr...,5
2,We have been going to Flying Fajita Sisters si...,4
3,Not a bad place for the first time diner. The ...,4
4,Had a wonderful meal here with a group of frie...,4
5,One cuisine I was told I wouldn't be able to f...,5
6,"hello, what kind of Mexican restaurant serve n...",1
7,The Flying Fajita Sistas has recently undergon...,4
8,Yum and wow!I had walked past this funky looki...,5
9,In a city lacking in the Mexican food departme...,4
